#  Unit 2.4a For Tri2 Project
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?
1. Flask app object
    Ive seen this working in the CPT projects in flask
2. SQLAlchemy object
    This is in the model and the API

In [3]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""
database = 'sqlite:///sqlite.db'  # path and filename of database
# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db' # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class
- class User purpose
    - The purpose of a "User" class depends on the specific software application or system that it is being used in. In general, a User class represents an individual who is interacting with the system, and it typically includes attributes such as the user's name, email, password, and other information that is relevant to the application.
- db.Model inheritance
    - In Flask and SQLAlchemy, you can create a hierarchy of classes using inheritance to model related database tables. When you define a model in SQLAlchemy, you can create a new model that inherits from an existing model by specifying the base model class as a parameter to the new class.
- _init_ method
    - the *__init__* method is a special method in Python classes that gets called when an instance of the class is created. It is also known as the constructor method.

The purpose of the *__init__* method is to initialize the attributes of the object to their initial state. This is where you can set default values for attributes, and accept arguments that can be used to initialize the object's attributes.

Here is an example of a simple class with an __ini
- @property, @<column>.setter
- additional methods

In [4]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL

class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts email from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows name to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter versus stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, to avoid unfriendly outcomes
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob should be have verification for type date
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
1. Create All Tables from db Object
    All items are created using the DB object
2. User Object Constructors
    Object constructors create the specific columns and such
3. Try / Except 
    Try this... except if something happens, do this...


In [5]:
"""Database Creation and Testing """
import sqlite3


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob= datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Records exist uid toby, or error.
Records exist uid niko, or error.
Records exist uid lex, or error.
Records exist uid whit, or error.
Records exist uid indi, or error.
Records exist uid raven, or error.


In [10]:
import sqlite3

# Define a Book class
class Book:
    def __init__(self, title, author, year_published, isbn):
        self.title = title
        self.author = author
        self.year_published = year_published
        self.isbn = isbn

# Create a connection to the database
conn = sqlite3.connect('books.db')

# Create a cursor object
c = conn.cursor()

# Create a table to store books
c.execute('''CREATE TABLE books
             (title text, author text, year_published int, isbn text)''')

# Define some sample books
b1 = Book('To Kill a Mockingbird', 'Harper Lee', 1960, '9780061120084')
b2 = Book('1984', 'George Orwell', 1949, '9780451524935')
b3 = Book('Pride and Prejudice', 'Jane Austen', 1813, '9780486284736')

# Add the sample books to the database
books = [b1, b2, b3]
for book in books:
    c.execute("INSERT INTO books VALUES (?, ?, ?, ?)",
              (book.title, book.author, book.year_published, book.isbn))

# Save changes to the database
conn.commit()

# Close the connection to the database
conn.close()


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
1. User.query.filter_by
    - This function will essentially filter out all of the user ids and will be a check for any specific calls made in the db
2. user.password
    - Creates password?? (Unknown, wasn't here)

In [54]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
1. user.find_by_uid() and try/except
    - this command tries to find user based on the ID. This should theoretically increase the ease of access when accesses large database files and trying to locate specifc information.
2. user = User(...)
    - This command should defind the user as whatever the input is. 
3. user.dob and try/except
4. user.create() and try/except
    - Creates a specific user if all traits are qualified.

## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
1. User.query.all
2. json_ready assignment

User.query.all() is a SQLAlchemy query that retrieves all instances of the User model from the database. In this code, it retrieves all users from the database.

json_ready is a list comprehension that creates a new list by iterating through each user in the table (which contains all users from the database) and calling the read() method on each user. The read() method of each User object would be responsible for creating a JSON representation of that object. This list of JSON objects is assigned to the json_ready variable, which is then returned by the read() function.

In [55]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # each user adds user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Add Update functionality to this blog.
- Add Delete functionality to this blog.

In [20]:
import sqlite3
database = 'instance/sqlite.db' 

def delete():
    # Get the id of the row to delete
    row_id = input("Enter the id of the row to delete: ")

    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    try:
        # Execute the DELETE command using the id column
        cursor.execute("DELETE FROM users WHERE id = ?", (row_id,))

        if cursor.rowcount == 0:
            # The row with the given id was not found in the table
            print(f"No row with id {row_id} was found in the table")
        else:
            # The row was successfully deleted
            print(f"The row with id {row_id} was successfully deleted from the table")
    except sqlite3.Error as error:
        print(f"Error deleting row: {error}")
    finally:
        # Commit the changes and close the database connection
        conn.commit()
        conn.close()
        
        
delete()


The row with id 3 was successfully deleted from the table


In [18]:
def update():
    uid=input("Enter user id")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass
password=input("Enter password")
destination = input("Update destination:")

update()

NameError: name 'find_by_uid' is not defined